In [109]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.functions import col,  udf, explode
import re

import random
import zipfile


In [110]:
spark = SparkSession.builder \
    .appName("data cgv") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g")\
    .getOrCreate()


In [111]:
schema_type = StructType([
    StructField("theater", StringType(), True),
    StructField("name_movie", StringType(), True),
    StructField("date", StringType(), True), 
    StructField("times", ArrayType(StringType()), True)
])

In [112]:
path = "../data/cgv_movies.json"
df = spark.read.option("multiLine", True).schema(schema_type).json(path)

df.show()

+--------------------+--------------------+-----------+--------------------+
|             theater|          name_movie|       date|               times|
+--------------------+--------------------+-----------+--------------------+
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔ...|09\nWed\n24|[19:20\n20:10\n20...|
|CGV Hung Vuong Plaza|          MƯA ĐỎ T13|09\nWed\n24|      [20:20\n22:50]|
|CGV Hung Vuong Plaza|THE CONJURING: LA...|09\nWed\n24|[19:40\n22:30, 21...|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔ...|09\nThu\n25|[08:10\n09:00\n09...|
|CGV Hung Vuong Plaza|CHAINSAW MAN - TH...|09\nThu\n25|             [18:00]|
|CGV Hung Vuong Plaza|          MƯA ĐỎ T13|09\nThu\n25|[08:30\n13:50\n16...|
|CGV Hung Vuong Plaza|THE CONJURING: LA...|09\nThu\n25|[08:00\n13:50\n19...|
|CGV Hung Vuong Plaza|BTS 2016 LIVE THE...|09\nThu\n25|      [14:10\n18:00]|
|CGV Hung Vuong Plaza|DEMON SLAYER: INF...|09\nThu\n25|      [10:50\n16:40]|
|CGV Hung Vuong Plaza|   THE LONG WALK T18|09\nThu\n25|             [11:55]|

In [113]:
def clean_date(date_str):
    date_list = re.findall(r'\d+', date_str)
    day, month = date_list[::-1]

    return (day, month)

clean_date_udf = udf(
                clean_date,
                StructType([
                    StructField("day", StringType(), True),
                    StructField("month", StringType(), True),
]))



def clean_time(times):
    list_time = []
    for time in times:
        list_time.extend(time.split("\n"))
    return list_time

clean_time_udf = udf(
    clean_time,
    ArrayType(StringType())
)


In [ ]:
df_final = (
    df.withColumn("date_new", clean_date_udf(col("date")))
      .withColumn("times_clean", clean_time_udf(col("times")))
      .select(
          "theater",
          "name_movie",
          col("date_new.day").alias("day"),
          col("date_new.month").alias("month"),
          "times_clean"
      )
)


+--------------------+---------------------------------------------------------------------------------+---+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|theater             |name_movie                                                                       |day|month|times_clean                                                                                                                                                                                                                            |
+--------------------+---------------------------------------------------------------------------------+---+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [115]:

df_exploded = df_final.withColumn("time", explode(col("times_clean"))) \
                      .drop("times_clean")  # bỏ cột cũ nếu muốn

df_exploded.show(truncate=False)

+--------------------+-----------------------------+---+-----+-----+
|theater             |name_movie                   |day|month|time |
+--------------------+-----------------------------+---+-----+-----+
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |19:20|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |20:10|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |20:40|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |21:30|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |21:50|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |22:40|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |23:00|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |21:10|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |23:30|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |19:50|
|CGV Hung Vuong Plaza|TỬ CHIẾN TRÊN KHÔNG T16      |24 |09   |22:20|
|CGV Hung Vuong Plaza|MƯA ĐỎ T13  